<a href="https://colab.research.google.com/github/mirovix/CV_final_project/blob/main/NN_boats.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import json
import io
from google.colab import drive
from google.colab import files
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.preprocessing.image import img_to_array
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import numpy as np
import glob
import cv2
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt

np.random.seed(42)

In [ ]:
drive.mount('/content/drive')

with open('/content/drive/MyDrive/dataset_boats/label.json') as json_file:
    json_file = json.load(json_file)

#print(len(json_file[30]['Label']))

x_start, y_start, x_end, y_end = [],[],[],[]
data,targets = [],[]
width = 32
height = 32

for row in json_file:

  if(len(row['Label']) == 0):
    continue

  x_start = float(row['Label']['objects'][0]['bbox']['left'])
  y_start = float(row['Label']['objects'][0]['bbox']['top'])
  x_end = float(row['Label']['objects'][0]['bbox']['left'] + row['Label']['objects'][0]['bbox']['width'])
  y_end = float(row['Label']['objects'][0]['bbox']['top'] + row['Label']['objects'][0]['bbox']['height'])
  
  image_path = "/content/drive/MyDrive/dataset_boats/train/" + row['External ID']
  cv_image = cv2.imread(image_path)

  x_start = (width * x_start) / cv_image.shape[1]
  y_start = (height * y_start) / cv_image.shape[0]
  x_end = (width * x_end) / cv_image.shape[1]
  y_end = (height * y_end) / cv_image.shape[0]
  targets.append((x_start, y_start, x_end, y_end))

  cv_image = cv2.resize(cv_image, (height, width))
  data.append(img_to_array(cv_image))
  


print(len(data))

In [ ]:
print(cv_image.shape[0])
print(cv_image.shape[1])


In [ ]:
# convert the data and targets to NumPy arrays, scaling the input
# pixel intensities from the range [0, 255] to [0, 1]
#data = np.array(data, dtype="float32") / 255.0
#targets = np.array(targets, dtype="float32")
# partition the data into training and testing splits using 90% of
# the data for training and the remaining 10% for testing
data = np.array(data, dtype="float32") / 255
targets = np.array(targets, dtype="float32")

X_train, X_test, y_train, y_test = train_test_split(data, targets, test_size=0.15, random_state=42)
X_train, X_val, y_train, y_val  = train_test_split(X_train, y_train, test_size=0.47, random_state=42) # 0.25 x (1 - 0.15) = 0.2125

In [ ]:
mean = np.mean(X_train, axis = 0)
std = np.std(X_train, axis = 0)

X_train = (X_train - mean) /  std
X_val = (X_val - mean) /  std
X_test = (X_test - mean) /  std

In [ ]:
def plot_loss(history):
  plt.figure(figsize=(10,6))
  plt.plot(history.epoch,history.history['loss'])
  plt.plot(history.epoch,history.history['val_loss'])
  plt.title('loss')

def plot_accuracy(history):
  plt.figure(figsize=(10,6))
  plt.plot(history.epoch,history.history['accuracy'])
  plt.plot(history.epoch,history.history['val_accuracy'])
  plt.title('accuracy')

In [ ]:
model = keras.models.Sequential([
    keras.layers.Conv2D(filters=128, kernel_size=[3,3], activation='relu', input_shape=[height, width, 3]),
    keras.layers.MaxPool2D(pool_size=[2,2]),
    keras.layers.Conv2D(filters=64, kernel_size=[2,2], activation='relu'),
    keras.layers.MaxPool2D(pool_size=[2,2]),
    keras.layers.Conv2D(filters=32, kernel_size=[2,2], activation='relu'),
    keras.layers.MaxPool2D(pool_size=[2,2]),
    keras.layers.Conv2D(filters=16, kernel_size=[2,2], activation='relu'),
    keras.layers.Flatten(),
    keras.layers.Dense(4, activation="sigmoid")
  ])

model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])
  
print(model.summary())

history = model.fit(X_train, y_train, epochs=50, batch_size=2 ,validation_data=(X_val, y_val))

plot_loss(history)
plot_accuracy(history)

scores = model.evaluate(X_test, y_test, verbose=2)
print(" %s: %.2f%%" % (model.metrics_names[1], scores[1]*100))
print("----------------------------")